In [1]:
import torch, torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import time
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import os
import random
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [2]:
#Зафиксируем seed для воспроизводимости
def seed_everything(seed):
    random.seed(seed) # фиксируем генератор случайных чисел
    os.environ['PYTHONHASHSEED'] = str(seed) # фиксируем заполнения хешей
    np.random.seed(seed) # фиксируем генератор случайных чисел numpy
    torch.manual_seed(seed) # фиксируем генератор случайных чисел pytorch

In [3]:
seed_everything(42)

In [21]:
# Applying Transforms to the Data
image_transforms = { 
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

In [22]:
# Load the Data

# Set train and valid directory paths

dataset = 'cifar10'

train_directory = os.path.join(dataset, 'train')
valid_directory = os.path.join(dataset, 'valid')

# Batch size
bs = 32

# Number of classes
num_classes = 10

# Load Data from folders
data = {
    'train': torchvision.datasets.CIFAR10(root="./cifar", train=True, download=True, transform=image_transforms['train']),
    'valid': torchvision.datasets.CIFAR10(root="./cifar", train=False, download=True, transform=image_transforms['valid']),
}

#Get a mapping of the indices to the class names, in order to see the output classes of the test images.
idx_to_class = {v: k for k, v in data['train'].class_to_idx.items()}
print(idx_to_class)

# Size of Data, to be used for calculating Average Loss and Accuracy
train_data_size = len(data['train'])
valid_data_size = len(data['valid'])

# Create iterators for the Data loaded using DataLoader module
train_data_loader = DataLoader(data['train'], batch_size=bs, shuffle=True)
valid_data_loader = DataLoader(data['valid'], batch_size=bs, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified
{0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

MobileNet 3й версии, Small.

In [6]:
mobilenet=models.mobilenetv3.mobilenet_v3_small(weights='DEFAULT')
mobilenet=mobilenet.to(device)

In [7]:
# Freeze model parameters
for param in mobilenet.parameters():
    param.requires_grad = False

In [8]:
mobilenet

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [10]:
num_classes = 10

In [11]:
# Change the final layer of mobilenet Model for Transfer Learning
fc_inputs = mobilenet.classifier[0].in_features

mobilenet.classifier = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.Hardswish(),
    nn.Dropout(0.2),
    nn.Linear(256, num_classes), # Since 10 possible outputs
    nn.LogSoftmax(dim=1) # For using NLLLoss()
)

# Convert model to be used on GPU
mobilenet = mobilenet.to(device)

In [12]:
# Define Optimizer and Loss Function
loss_func = nn.NLLLoss()
optimizer = optim.Adam(mobilenet.parameters(),lr=1e-3)

In [17]:
def train_and_validate_1(model, loss_criterion, optimizer, epochs=3):
    
    start = time.time()
    history = []
    best_loss = 100000.0
    best_epoch = None

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))
        
        # Set to training mode
        model.train()
        
        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0
        
        valid_loss = 0.0
        valid_acc = 0.0
        
        #correct_5=0.0
        correct=0
        c=0
        total=0
        
        for i, (inputs, labels) in enumerate(train_data_loader):

            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Clean existing gradients
            optimizer.zero_grad()
            
            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)
            
            # Compute loss
            loss = loss_criterion(outputs, labels)
            
            # Backpropagate the gradients
            loss.backward()
            
            # Update the parameters
            optimizer.step()
            
            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item() * inputs.size(0)
            
            # Compute the accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            
            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            
            # Compute total accuracy in the whole batch and add to train_acc
            train_acc += acc.item() * inputs.size(0)
            
#             print("Batch number: {:03d}, Valid: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))

        
        #Validation - No gradient tracking needed
        with torch.no_grad():

            # Set to evaluation mode
            model.eval()

            # Validation loop
            for j, (inputs, labels) in enumerate(valid_data_loader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass - compute outputs on input data using the model
                outputs = model(inputs)

                # Compute loss
                loss = loss_criterion(outputs, labels)

                # Compute the total loss for the batch and add it to valid_loss
                valid_loss += loss.item() * inputs.size(0)

                # Calculate validation accuracy
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                # Convert correct_counts to float and then compute the mean
                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                # Compute total accuracy in the whole batch and add to valid_acc
                valid_acc += acc.item() * inputs.size(0)
                
#                 # Compute top-5 accuracy
#                 total+=labels.size(0)
#                 c+=(predictions==labels).sum().item()
#                 pred=outputs.topk(4,largest=True,sorted=True)[0]
#                 lab=labels.view(labels.size(0),-1).expand_as(pred)
#                 correct=pred.eq(lab).float()
#                 correct_5+=correct[:,:4].sum()
                
                #print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))
        
        if valid_loss < best_loss:
            best_loss = valid_loss
            best_epoch = epoch

        #Find average training loss and training accuracy
        avg_train_loss = train_loss/train_data_size 
        avg_train_acc = train_acc/train_data_size

        # Find average training loss and training accuracy
        avg_valid_loss = valid_loss/valid_data_size 
        avg_valid_acc = valid_acc/valid_data_size

        history.append([avg_train_loss, avg_train_acc, avg_valid_loss, avg_valid_acc])
                
        epoch_end = time.time()
    
        print("Epoch : {:03d}, Training: Loss - {:.4f}, Accuracy - {:.4f}%, \n\t\tValidation : Loss - {:.4f}, Accuracy - {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))
#         print('Top 5 error:%2.2f' % (1-correct_5/total))        
#         # Save if the model has best accuracy till now
#         torch.save(model, dataset+'_model_'+str(epoch)+'.pt')
            
    return model, history, best_epoch

In [23]:
# Train the model for num epochs:
num_epochs = 5
trained_model, history, best_epoch = train_and_validate_1(mobilenet, loss_func, optimizer, num_epochs)

Epoch: 1/5
Epoch : 000, Training: Loss - 0.7102, Accuracy - 75.1960%, 
		Validation : Loss - 0.5939, Accuracy - 79.0800%, Time: 152.0817s
Epoch: 2/5
Epoch : 001, Training: Loss - 0.6717, Accuracy - 76.5800%, 
		Validation : Loss - 0.5898, Accuracy - 79.6100%, Time: 150.2612s
Epoch: 3/5
Epoch : 002, Training: Loss - 0.6516, Accuracy - 77.3760%, 
		Validation : Loss - 0.6223, Accuracy - 78.3900%, Time: 149.9488s
Epoch: 4/5
Epoch : 003, Training: Loss - 0.6446, Accuracy - 77.5220%, 
		Validation : Loss - 0.5346, Accuracy - 81.6100%, Time: 149.0214s
Epoch: 5/5
Epoch : 004, Training: Loss - 0.6318, Accuracy - 78.0500%, 
		Validation : Loss - 0.5329, Accuracy - 81.3600%, Time: 149.4360s


In [24]:
class Cifar10SearchDataset(torchvision.datasets.CIFAR10):
    def __init__(self, root="~/data/cifar10", train=True, download=True, transform=None):
        super().__init__(root=root, train=train, download=download, transform=transform)

    def __getitem__(self, index):
        image, label = self.data[index], self.targets[index]

        if self.transform is not None:
            transformed = self.transform(image=image)
            image = transformed["image"]

        return image, label

In [25]:
train_transform = A.Compose([
        A.RandomResizedCrop(224,224),
        A.HorizontalFlip(p=0.5),
        A.RandomGamma(gamma_limit=(80, 120), eps=None, always_apply=False, p=0.5),
        A.RandomBrightnessContrast (p=0.5),
        A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), always_apply=False, p=0.5),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ToTensorV2()
    ])



val_transform = A.Compose([
        A.Resize(256,256),
        A.CenterCrop(224,224),
        A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ToTensorV2()
    ])

In [26]:
cifar_train = Cifar10SearchDataset(
root="./cifar", train=True, download=True, transform=train_transform
)

cifar_val = Cifar10SearchDataset(
root="./cifar", train=False, download=True, transform=val_transform
)

train_dataloader = torch.utils.data.DataLoader(
cifar_train, batch_size=32, shuffle=True, num_workers=4
)

val_dataloader = torch.utils.data.DataLoader(
cifar_val, batch_size=32, shuffle=False, num_workers=4
)

Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [27]:
train_num = len(cifar_train)
val_num=len(cifar_val)

In [28]:
print("using {} images for training, {} images for  validation.".format(train_num, val_num))

using 50000 images for training, 10000 images for  validation.


In [29]:
def train_and_validate(model, loss_criterion, optimizer, epochs=3):
    
    start = time.time()
    history = []
    best_loss = 100000.0
    best_epoch = None

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))
        
        # Set to training mode
        model.train()
        
        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0
        
        valid_loss = 0.0
        valid_acc = 0.0
        
        #correct_5=0.0
        correct=0
        c=0
        total=0
        
        for i, (inputs, labels) in enumerate(train_dataloader):

            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Clean existing gradients
            optimizer.zero_grad()
            
            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)
            
            # Compute loss
            loss = loss_criterion(outputs, labels)
            
            # Backpropagate the gradients
            loss.backward()
            
            # Update the parameters
            optimizer.step()
            
            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item() * inputs.size(0)
            
            # Compute the accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            
            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            
            # Compute total accuracy in the whole batch and add to train_acc
            train_acc += acc.item() * inputs.size(0)
            
#             print("Batch number: {:03d}, Valid: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))

        
        #Validation - No gradient tracking needed
        with torch.no_grad():

            # Set to evaluation mode
            model.eval()

            # Validation loop
            for j, (inputs, labels) in enumerate(val_dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass - compute outputs on input data using the model
                outputs = model(inputs)

                # Compute loss
                loss = loss_criterion(outputs, labels)

                # Compute the total loss for the batch and add it to valid_loss
                valid_loss += loss.item() * inputs.size(0)

                # Calculate validation accuracy
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                # Convert correct_counts to float and then compute the mean
                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                # Compute total accuracy in the whole batch and add to valid_acc
                valid_acc += acc.item() * inputs.size(0)
                
#                 # Compute top-5 accuracy
#                 total+=labels.size(0)
#                 c+=(predictions==labels).sum().item()
#                 pred=outputs.topk(4,largest=True,sorted=True)[0]
#                 lab=labels.view(labels.size(0),-1).expand_as(pred)
#                 correct=pred.eq(lab).float()
#                 correct_5+=correct[:,:4].sum()
                
                #print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))
        
        if valid_loss < best_loss:
            best_loss = valid_loss
            best_epoch = epoch

        #Find average training loss and training accuracy
        avg_train_loss = train_loss/train_num 
        avg_train_acc = train_acc/train_num

        # Find average training loss and training accuracy
        avg_valid_loss = valid_loss/val_num 
        avg_valid_acc = valid_acc/val_num

        history.append([avg_train_loss, avg_train_acc, avg_valid_loss, avg_valid_acc])
                
        epoch_end = time.time()
    
        print("Epoch : {:03d}, Training: Loss - {:.4f}, Accuracy - {:.4f}%, \n\t\tValidation : Loss - {:.4f}, Accuracy - {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))
#         print('Top 5 error:%2.2f' % (1-correct_5/total))        
#         # Save if the model has best accuracy till now
#         torch.save(model, dataset+'_model_'+str(epoch)+'.pt')
            
    return model, history, best_epoch

In [30]:
# Train the model for num epochs:
num_epochs = 5
trained_model, history, best_epoch = train_and_validate(mobilenet, loss_func, optimizer, num_epochs)

Epoch: 1/5
Epoch : 000, Training: Loss - 1.1379, Accuracy - 60.1640%, 
		Validation : Loss - 0.5645, Accuracy - 80.6000%, Time: 186.7543s
Epoch: 2/5
Epoch : 001, Training: Loss - 1.0925, Accuracy - 61.7760%, 
		Validation : Loss - 0.5694, Accuracy - 80.1100%, Time: 186.9936s
Epoch: 3/5
Epoch : 002, Training: Loss - 1.0786, Accuracy - 61.7580%, 
		Validation : Loss - 0.5602, Accuracy - 80.5200%, Time: 187.6432s
Epoch: 4/5
Epoch : 003, Training: Loss - 1.0711, Accuracy - 62.6140%, 
		Validation : Loss - 0.5714, Accuracy - 80.5200%, Time: 193.6202s
Epoch: 5/5
Epoch : 004, Training: Loss - 1.0625, Accuracy - 62.8140%, 
		Validation : Loss - 0.5681, Accuracy - 80.9300%, Time: 193.2489s


In [ ]:
!pip install -q -U albumentations
!echo 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 5.7 MB/s eta 0:00:00



In [ ]:
! pip install --upgrade tensorflow_datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Попробую Torchvision, а именно AutoAugment с baseline configuration.


In [13]:
from torchvision.transforms.autoaugment import AutoAugmentPolicy

In [14]:
# Applying Transforms to the Data
image_transforms_t = { 
    'train': transforms.Compose([
             transforms.Resize(size=256),
             transforms.CenterCrop(size=224),
             transforms.AutoAugment(AutoAugmentPolicy.CIFAR10),
             transforms.ToTensor(),
             transforms.Normalize(mean=[0.485, 0.456, 0.406],
             std=[0.229, 0.224, 0.225]),
                
                
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

In [15]:
# Load the Data

# Set train and valid directory paths

dataset = 'cifar10'

train_directory = os.path.join(dataset, 'train')
valid_directory = os.path.join(dataset, 'valid')

# Batch size
bs = 32

# Number of classes
num_classes = 10

# Load Data from folders
data = {
    'train': torchvision.datasets.CIFAR10(root="./cifar", train=True, download=True, transform=image_transforms_t['train']),
    'valid': torchvision.datasets.CIFAR10(root="./cifar", train=False, download=True, transform=image_transforms_t['valid']),
}

#Get a mapping of the indices to the class names, in order to see the output classes of the test images.
idx_to_class = {v: k for k, v in data['train'].class_to_idx.items()}
print(idx_to_class)

# Size of Data, to be used for calculating Average Loss and Accuracy
train_data_size = len(data['train'])
valid_data_size = len(data['valid'])

# Create iterators for the Data loaded using DataLoader module
train_data_loader = DataLoader(data['train'], batch_size=bs, shuffle=True)
valid_data_loader = DataLoader(data['valid'], batch_size=bs, shuffle=True)

100%|██████████| 170498071/170498071 [00:05<00:00, 29677308.21it/s]


Extracting ./cifar/cifar-10-python.tar.gz to ./cifar
Files already downloaded and verified
{0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}


In [18]:
# Train the model for num epochs:
num_epochs = 5
trained_model, history, best_epoch = train_and_validate_1(mobilenet, loss_func, optimizer, num_epochs)

Epoch: 1/5
Epoch : 000, Training: Loss - 0.9507, Accuracy - 66.8540%, 
		Validation : Loss - 0.5201, Accuracy - 81.6700%, Time: 169.0958s
Epoch: 2/5
Epoch : 001, Training: Loss - 0.7995, Accuracy - 71.7980%, 
		Validation : Loss - 0.4471, Accuracy - 84.3500%, Time: 163.0915s
Epoch: 3/5
Epoch : 002, Training: Loss - 0.7624, Accuracy - 73.1560%, 
		Validation : Loss - 0.4342, Accuracy - 84.5900%, Time: 164.0158s
Epoch: 4/5
Epoch : 003, Training: Loss - 0.7373, Accuracy - 74.1300%, 
		Validation : Loss - 0.3981, Accuracy - 86.2400%, Time: 161.6811s
Epoch: 5/5
Epoch : 004, Training: Loss - 0.7191, Accuracy - 74.8480%, 
		Validation : Loss - 0.4097, Accuracy - 85.8000%, Time: 161.7454s


In [31]:
import pandas as pd
df = pd.DataFrame({'models': ['baseline transforms', 'albumentations', 'torchvision'], 
                   'accuracy, %': [81.36, 80.93 , 85.8] 
                   })

In [32]:
df

,models,"accuracy, %"
0,baseline transforms,81.36
1,albumentations,80.93
2,torchvision,85.80
